1. Load the packages

In [ ]:
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot
import torch
import torch.nn as nn
from torch.autograd.function import Function
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from  torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler

2. Define the function used to load MNIST and USPS data splits



You need to download[ MNIST-USPS data splits](https://github.com/samotiian/CCSA) generated in [1] to run this code. Then: 


2.1. If you run the code on Colab, you will need to put these splits in the corresponding folder of your [Google Drive](https://drive.google.com/drive/u/0/my-drive).


2.2. If you run the code locally, you will need to put these splits in the corresponding folder of your device.



[1] Motiian, S., Piccirilli, M., Adjeroh, D. A., & Doretto, G. (2017). Unified deep supervised domain adaptation and generalization. In Proceedings of the IEEE international conference on computer vision (pp. 5715-5725).

In [ ]:
# Mount the Google drive, please ignore this cell if you run the code locally.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# The path where you put the data.
initial_path='/content/drive/MyDrive/MINIST domain adaptation/CCSA-master/row_data/'

In [ ]:
# The function used to load the data, refered to:
##
#Motiian, S., Piccirilli, M., Adjeroh, D. A., & Doretto, G. (2017). 
#Unified deep supervised domain adaptation and generalization. 
#In Proceedings of the IEEE international conference on computer vision (pp. 5715-5725).
##


def read_data(domain_adaptation_task,repetition,sample_per_class):
    UM  = domain_adaptation_task
    cc  = repetition
    SpC = sample_per_class
    if UM != 'MNIST_to_USPS':
        if UM != 'USPS_to_MNIST':
            raise Exception('domain_adaptation_task should be either MNIST_to_USPS or USPS_to_MNIST')


    if cc <0 or cc>10:
        raise Exception('number of repetition should be between 0 and 9.')

    if SpC <1 or SpC>7:
            raise Exception('number of sample_per_class should be between 1 and 7.')

    X_train_target=np.load(initial_path + UM + '_X_train_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')
    y_train_target=np.load(initial_path + UM + '_y_train_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')

    X_train_source=np.load(initial_path + UM + '_X_train_source_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')
    y_train_source=np.load(initial_path + UM + '_y_train_source_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')

    X_test = np.load(initial_path + UM + '_X_test_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC)+'.npy')
    y_test = np.load(initial_path + UM + '_y_test_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC)+'.npy')
    

    
    return X_train_target,y_train_target,X_train_source,y_train_source,X_test,y_test

3. Self-defined layer for implementing Center Transfer loss (CTL)

In [ ]:
class CTL(nn.Module):
    def __init__(self, num_classes, feat_dim, size_average=True):
        super(CTL, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_classes, feat_dim))
        self.centerlossfunc = CenterlossFunc.apply
        self.feat_dim = feat_dim
        self.size_average = size_average

    def forward(self, label, flip_label, domain_label,feat):
        batch_size = feat.size(0)
        feat = feat.view(batch_size, -1)
        # To check the dim of centers and features
        if feat.size(1) != self.feat_dim:
            raise ValueError("Center's dim: {0} should be equal to input feature's \
                            dim: {1}".format(self.feat_dim,feat.size(1)))
        batch_size_tensor = feat.new_empty(1).fill_(batch_size if self.size_average else 1)
        loss = self.centerlossfunc(feat, label,flip_label, domain_label,self.centers, batch_size_tensor)
        return loss


class CenterlossFunc(Function):
    @staticmethod
    def forward(ctx, feature, label,flip_label, domain_label,centers, batch_size):
        ctx.save_for_backward(feature, label,flip_label, domain_label,centers, batch_size)
        centers_batch = centers.index_select(0, flip_label.long())

        domain_counts = centers.new_zeros(2)
        domain_ones = centers.new_ones(domain_label.size(0))
        domain_counts = domain_counts.scatter_add_(0, domain_label.long(), domain_ones)+0.0001
        domain_counts = torch.index_select(domain_counts, 0, domain_label.int())  

        return ((feature - centers_batch).pow(2)/domain_counts.view(-1, 1)).sum() / 2.0 / batch_size

    @staticmethod
    def backward(ctx, grad_output):
        feature,label,flip_label,domain_label,centers,batch_size = ctx.saved_tensors
        flip_centers_batch = centers.index_select(0, flip_label.long())
        flip_diff=flip_centers_batch-feature

        domain_counts = centers.new_zeros(2)
        domain_ones = centers.new_ones(domain_label.size(0))
        domain_counts = domain_counts.scatter_add_(0, domain_label.long(), domain_ones)+0.0001
        domain_counts = torch.index_select(domain_counts, 0, domain_label.int())  
      


        centers_batch = centers.index_select(0, label.long())
        diff = centers_batch - feature
        # init every iteration
        counts = centers.new_ones(centers.size(0))
        ones = centers.new_ones(label.size(0))
        grad_centers = centers.new_zeros(centers.size())
        counts = counts.scatter_add_(0, label.long(), ones)-1+0.001

        grad_centers.scatter_add_(0, label.unsqueeze(1).expand(feature.size()).long(), diff)
        grad_centers = grad_centers/counts.view(-1, 1)
       
        return - grad_output * (flip_diff/domain_counts.view(-1, 1)) / batch_size, None, None,None,grad_centers / batch_size, None

#Testing function
def main(test_cuda=False):
    print('-'*80)
    device = torch.device("cuda" if test_cuda else "cpu")
    ct = CTL(20,2,size_average=True).to(device)
    y = torch.Tensor([0,0,2,1,3]).to(device)
    dola= torch.Tensor([0,0,1,1,0]).to(device)
    feat = torch.zeros(5,2).to(device).requires_grad_()
    print(list(ct.parameters()))
    
    print(ct.centers.grad)
    out = ct(y,y,dola,feat)
    print(out.item())
    out.backward()


if __name__ == '__main__':
    torch.manual_seed(999)
    if torch.cuda.is_available():
        main(test_cuda=True)

--------------------------------------------------------------------------------
[Parameter containing:
tensor([[-0.2528,  1.4072],
        [ 0.2910,  1.0365],
        [-0.9816, -3.4219],
        [ 1.4910,  0.2422],
        [ 1.4832, -0.3704],
        [ 0.0941,  2.1528],
        [ 0.6271, -1.1666],
        [-0.7862,  0.0759],
        [-0.0086, -0.6568],
        [-1.0011,  0.2992],
        [ 0.6396, -1.0857],
        [-1.6153,  1.5635],
        [ 0.8194,  0.6117],
        [ 0.7602,  1.4788],
        [ 1.9647,  0.9414],
        [ 0.3883, -0.3957],
        [ 0.5920, -2.8563],
        [-0.4750, -0.9978],
        [ 0.0489,  0.9250],
        [-1.2278, -0.9470]], device='cuda:0', requires_grad=True)]
None
0.9038917422294617


4. Define LetNet++ for MNIST-USPS

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1_1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.prelu1_1 = nn.PReLU()
        self.conv1_2 = nn.Conv2d(32, 32, kernel_size=5, padding=2)
        self.prelu1_2 = nn.PReLU()
        self.conv2_1 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.prelu2_1 = nn.PReLU()
        self.conv2_2 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.prelu2_2 = nn.PReLU()
        self.conv3_1 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.prelu3_1 = nn.PReLU()
        self.conv3_2 = nn.Conv2d(128, 128, kernel_size=5, padding=2)
        self.prelu3_2 = nn.PReLU()
        self.preluip1 = nn.PReLU()
        
        self.ip1 = nn.Linear(128*2*2, 84)
        self.ip2 = nn.Linear(84, 10, bias=False)


    def forward(self, x):
        x = self.prelu1_1(self.conv1_1(x))
        x = self.prelu1_2(self.conv1_2(x))
        x = F.max_pool2d(x,2)
        x = self.prelu2_1(self.conv2_1(x))
        x = self.prelu2_2(self.conv2_2(x))
        x = F.max_pool2d(x,2)
        x = self.prelu3_1(self.conv3_1(x))
        x = self.prelu3_2(self.conv3_2(x))
        x = F.max_pool2d(x,2)
        x = x.view(-1, 128*2*2)

        ip1 = self.preluip1(self.ip1(x))
        ip2 = self.ip2(ip1)
        return ip1, F.log_softmax(ip2, dim=1)

5. Define the training in one epoch

In [ ]:
#deine the training within 1 epoch

def train(epoch,loss_weight,train_loader,model,nllloss,centerloss,optimizer4nn,optimzer4center,x_test,y_test,device):
  center_total=0
  nll_losss=0
  #Training in each epoch#
  for data, target,good_ord,flip_ord,domain_label in train_loader:
    data, target,good_ord,flip_ord,domain_label = data.to(device), target.to(device),good_ord.to(device), flip_ord.to(device),domain_label.to(device)

    ip1, pred = model(data)
    loss = nllloss(pred, target) + loss_weight* centerloss(good_ord, flip_ord,domain_label,ip1)

    optimizer4nn.zero_grad()
    optimzer4center.zero_grad()

    loss.backward()

    optimizer4nn.step()
    optimzer4center.step()

    center_total=center_total+centerloss(good_ord, flip_ord,domain_label,ip1).item()
    nll_losss= nll_losss+nllloss(pred, target).item()*64
  test_out = model(x_test.to(device))[1]
  pred_y = torch.max(test_out.cpu(), 1)[1].data.numpy()
  accuracy = float((pred_y == y_test.data.numpy()).astype(int).sum()) / float(y_test.size(0))

  return accuracy



6. Define the training function

In [ ]:
#Activate GPU
def trainss(ministorusps,repetition,sample_per_class,weightss,center_step):

  use_cuda = torch.cuda.is_available() and True
  device = torch.device("cuda" if use_cuda else "cpu")

 
  repetition=repetition
  sample_per_class=sample_per_class
  # MNIST_to_USPS
  X_train_target,y_train_target,X_train_source,y_train_source,X_test,y_test=read_data(ministorusps,repetition,sample_per_class)
  

  domain_label=np.hstack((np.zeros(len(y_train_source)),np.ones(len(y_train_target))))

  x_train=np.vstack((X_train_source,X_train_target))
  y_train=np.hstack((y_train_source,y_train_target))
  
  good_order=np.hstack((y_train_source,y_train_target+10))
  flip_order=np.hstack((y_train_source+10,y_train_target))


  x_train=torch.from_numpy(x_train).reshape(-1,1,16,16)
  y_train=torch.from_numpy(y_train)
  x_test=torch.from_numpy(X_test).reshape(-1,1,16,16)
  y_test=torch.from_numpy(y_test)

  good_order=torch.from_numpy(good_order)
  flip_order=torch.from_numpy(flip_order)
  domain_label=torch.from_numpy(domain_label)



  x_train=x_train.float()
  x_test=x_test.float()
  y_train=y_train.long()
  y_test=y_test.long()

  good_order=good_order.long()
  flip_order=flip_order.long()
  domain_label=domain_label.long()
  # form the dataset
  train_dataset=TensorDataset(x_train,y_train,good_order,flip_order,domain_label)
  val_dataset=TensorDataset(x_test,y_test)



  train_loader= DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
  test_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=2)

  # Model
  model = Net().to(device)


  # Softmax
  nllloss = nn.CrossEntropyLoss().to(device) 

  # λ
  loss_weight =  weightss

  # CTL
  ctl = CTL(20, 84).to(device)

  # optimzer4nn
  optimizer4nn = optim.Adam(model.parameters(),lr=0.001)

  sheduler = lr_scheduler.StepLR(optimizer4nn,20,gamma=0.8)

  # optimzer4center
  optimzer4center = torch.optim.SGD(ctl.parameters(), lr =center_step) #α
  best=[0]
  eps=0
  for epoch in range(100):
    accuracy=train(epoch+1, loss_weight,train_loader,model,nllloss,ctl,optimizer4nn,optimzer4center,x_test,y_test,device)
    if accuracy > max(best):
      best.append(accuracy)
      eps=epoch

  print("Testing accuracy:", str(max(best)))
  return model


7. Sensitivity analysis on λ

Note that the result is slightly different from that displayed in the manuscript. The difference is due to the random initilization. You can still see the **stable performance** of the models. 

In [ ]:
Rep=1
sample_per_class=3
center_step=0.5
lambda_para_set=[0,0.1,0.25,0.5,0.75,1,2.5,5,7.5,10,100]
for i in lambda_para_set:
  print('='*20,'λ='+str(i),'='*20)
  model=trainss('USPS_to_MNIST',Rep,sample_per_class,i,center_step)

==================== λ=0 ====================
Testing accuracy: 0.826
==================== λ=0.1 ====================
Testing accuracy: 0.9095
==================== λ=0.25 ====================
Testing accuracy: 0.907
==================== λ=0.5 ====================
Testing accuracy: 0.9045
==================== λ=0.75 ====================
Testing accuracy: 0.919
==================== λ=1 ====================
Testing accuracy: 0.904
==================== λ=2.5 ====================
Testing accuracy: 0.914
==================== λ=5 ====================
Testing accuracy: 0.9185
==================== λ=7.5 ====================
Testing accuracy: 0.919
==================== λ=10 ====================
Testing accuracy: 0.9095
==================== λ=100 ====================
Testing accuracy: 0.914


7. Sensitivity analysis on α

In [ ]:
Rep=1
sample_per_class=3
center_step=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
lambda_para=0.75
for i in center_step:
  print('='*20,'α='+str(i),'='*20)
  model=trainss('USPS_to_MNIST',Rep,sample_per_class,lambda_para,i)

==================== α=0.01 ====================
Testing accuracy: 0.8345
==================== α=0.05 ====================
Testing accuracy: 0.872
==================== α=0.1 ====================
Testing accuracy: 0.92
==================== α=0.2 ====================
Testing accuracy: 0.9185
==================== α=0.3 ====================
Testing accuracy: 0.903
==================== α=0.4 ====================
Testing accuracy: 0.911
==================== α=0.5 ====================
Testing accuracy: 0.921
==================== α=0.6 ====================
Testing accuracy: 0.897
==================== α=0.7 ====================
Testing accuracy: 0.8945
==================== α=0.8 ====================
Testing accuracy: 0.921
==================== α=0.9 ====================
Testing accuracy: 0.9125
==================== α=1 ====================
Testing accuracy: 0.9
